## 1. Library Import

In [3]:


# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
#from tqdm import tqdm
import pickle
import requests
import time
#import missingno as msno
import warnings;warnings.filterwarnings('ignore')


# dataframe 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# import eli5
# from eli5.sklearn import PermutationImportance

## 2. Data Load

In [4]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_original = dt.copy()
dt_test_original = dt_test.copy()

In [5]:
# Train과 Test data를 살펴보겠습니다.
display(dt.head(1))
display(dt_test.head(1))      # 부동산 실거래가(=Target) column이 제외된 모습입니다.

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,언주로 3,NaN,,직거래,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0


#### 2.2 데이터 탐색
- 데이터 탐색에는 데이터의 크기, 변수, 변수 유형을 확인합니다.
- 변수 유형의 경우 데이터를 직접 구별하여 범주형, 숫자형, 시간형으로 분류합니다. 

In [6]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)
print("Train Test ratio:", dt_test.shape[0]/dt.shape[0])

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)
Train Test ratio: 0.008287287879573337


In [7]:
# Train data와 Test data 변수는 아래와 같습니다.
print(len(dt.columns),"가지의 변수들")
print(dt.columns)

print(len(dt_test.columns),"가지의 변수들")
print(dt_test.columns)

52 가지의 변수들
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')
51 가지의 변수들
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적'

## 3. Data Preprocessing

#### 3.1. 결측치 탐색 및 보간
- 본 데이터는 업스테이지에서 제공한 데이터로 변수에 대한 설명이 극히 제한적입니다. 변수에 대한 설명이 있었다면 결측치가 있는 변수를 제거하는 방식 외에 다른 보간법을 판단하기가 매우 어렵스럽습니다.
- 그리하여, 결측치가 반 이상인 변수(열) 삭제를 결정하였습니다. 
- 단 좌표X, 좌표Y의 경우 주소(시군구, 본번, 부번, 번지)를 활용하여 결측치를 모두 채울 수 있었습니다.


##### 3.1.1 좌표 결측치 보간
- 시군구와 도로명, 번지 조합, 혹은 시군구와 본번, 부번 조합으로 좌표를 구할 수 있습니다. 
- 아파트의 경우 같은 도로명에 있을 경우 대대분 같은 단지를 형성하여, 결측치가 적은 시군구와 도로명를 1순위로 좌표를 구합니다.
- 아래 시군구와 도로명 조합에서 결측치가 없음을 보입니다. 

In [8]:
print(dt["시군구"].isna().sum())
print(dt["도로명"].isna().sum())
print(dt["번지"].isna().sum())
print(dt["본번"].isna().sum())
print(dt["본번"].isna().sum())

0
0
225
75
75


- Train 데이터에는 좌표 결측 행이 869,670개, Test 데이터에는 6,562개가 존재합니다.
- 이 중 중복 주소를 제거한 고유 주소는 8,513개이며, 해당 주소들에 대해 카카오맵 API를 사용해 좌표를 생성합니다.
- 좌표 생성에 시군구와 도로명만을 사용합니다. 

In [9]:
def print_missing_coord_fallback_stats(train, test):

    def calc(df, name):
        # 0. 좌표가 하나라도 없는 행만
        missing = df[df["좌표X"].isna() | df["좌표Y"].isna()]
        total_missing = len(missing)

        # 1️. 시군구 + 도로명
        cond1 = (
            missing["시군구"].notna() &
            missing["도로명"].notna()
        )

        # 2️. 시군구 + 번지 (1번 실패한 것만)
        cond2 = (
            ~cond1 &
            missing["시군구"].notna() &
            missing["번지"].notna()
        )

        # 3️. 시군구 + 본번(-부번) (1,2번 실패한 것만)
        cond3 = (
            ~cond1 & ~cond2 &
            missing["시군구"].notna() &
            missing["본번"].notna()
        )

        # 전부 실패
        cond_fail = ~(cond1 | cond2 | cond3)

        print(f"\n[{name} 데이터 – 좌표 결측 행 기준 fallback 결과]")
        print(f"좌표 결측 행 수              : {total_missing:,}")
        print(f"1️. 시군구 + 도로명              : {cond1.sum():,}")
        print(f"2️. 시군구 + 번지                : {cond2.sum():,}")
        print(f"3️. 시군구 + 본번(-부번)         : {cond3.sum():,}")
        print(f"주소 생성 불가               : {cond_fail.sum():,}")

    calc(train, "Train")
    calc(test, "Test")

print_missing_coord_fallback_stats(dt, dt_test)



[Train 데이터 – 좌표 결측 행 기준 fallback 결과]
좌표 결측 행 수              : 869,670
1️. 시군구 + 도로명              : 869,670
2️. 시군구 + 번지                : 0
3️. 시군구 + 본번(-부번)         : 0
주소 생성 불가               : 0

[Test 데이터 – 좌표 결측 행 기준 fallback 결과]
좌표 결측 행 수              : 6,562
1️. 시군구 + 도로명              : 6,562
2️. 시군구 + 번지                : 0
3️. 시군구 + 본번(-부번)         : 0
주소 생성 불가               : 0


In [ ]:
import pandas as pd
import requests
import time
import re

KAKAO_API_KEY = "카카오 API 키 입력"
HEADERS = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
URL = "https://dapi.kakao.com/v2/local/search/address.json"


def kakao_geocode(addr):
    r = requests.get(URL, headers=HEADERS, params={"query": addr}, timeout=5)
    if r.status_code != 200:
        return None
    docs = r.json().get("documents", [])
    if not docs:
        return None
    return float(docs[0]["x"]), float(docs[0]["y"])


def normalize_addr(s):
    if pd.isna(s):
        return None
    s = str(s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def print_progress(current, total, bar_len=30):
    percent = current / total
    filled = int(bar_len * percent)
    bar = "█" * filled + "-" * (bar_len - filled)
    print(f"\r|{bar}| {percent*100:6.2f}%", end="")

def remove_dong(sigungu):
    if pd.isna(sigungu):
        return None
    parts = sigungu.split()
    return " ".join([p for p in parts if not p.endswith("동")])

# def fill_by_key(df, key_cols, cache, step_name, transform=None):
#     mask = df["좌표X"].isna() | df["좌표Y"].isna()

#     sub = df.loc[mask, key_cols].dropna().drop_duplicates()

#     if transform:
#         sub = sub.assign(addr=sub.apply(transform, axis=1))
#     else:
#         sub = sub.assign(
#             addr=sub[key_cols].astype(str).agg(" ".join, axis=1)
#         )

#     sub["addr"] = sub["addr"].map(normalize_addr)
#     addrs = sub["addr"].dropna().unique()

#     print(f"\n[{step_name}]")
#     print(f"- 처리 대상 고유 주소 수 : {len(addrs):,}")

#     for i, addr in enumerate(addrs, 1):
#         if addr in cache:
#             continue
#         cache[addr] = kakao_geocode(addr)
#         time.sleep(0.05)
#         if i % 10 == 0 or i == len(addrs):
#             print_progress(i, len(addrs))

#     print()

#     df.loc[mask, "addr_tmp"] = (
#         df.loc[mask, key_cols]
#         .astype(str)
#         .agg(" ".join, axis=1)
#         .map(normalize_addr)
#     )

#     df.loc[mask, ["좌표X", "좌표Y"]] = (
#         df.loc[mask, "addr_tmp"]
#         .map(cache)
#         .apply(lambda x: pd.Series(x) if x else pd.Series([None, None]))
#         .values
#     )

#     df.drop(columns="addr_tmp", inplace=True)

#     return df

def fill_by_key(df, key_cols, cache, step_name, transform=None):
    mask = df["좌표X"].isna() | df["좌표Y"].isna()

    sub = df.loc[mask, key_cols].dropna().drop_duplicates()

    if transform:
        sub = sub.assign(addr=sub.apply(transform, axis=1))
    else:
        sub = sub.assign(
            addr=sub[key_cols].astype(str).agg(" ".join, axis=1)
        )

    sub["addr"] = sub["addr"].map(normalize_addr)
    addrs = sub["addr"].dropna().unique()
    total = len(addrs)

    print(f"\n[{step_name}]")
    print(f"- 처리 대상 고유 주소 수 : {total:,}")

    if total == 0:
        return df

    # ===== 예상 시간 계산 (probe) =====
    probe_n = min(100, total)
    probe_start = time.perf_counter()

    for addr in addrs[:probe_n]:
        if addr not in cache:
            cache[addr] = kakao_geocode(addr)
            time.sleep(0.05)

    probe_elapsed = time.perf_counter() - probe_start
    est_total_time = probe_elapsed / probe_n * total

    print(f"- 예상 소요 시간 : {est_total_time/60:.1f} 분")

    # ===== 본 처리 =====
    for i, addr in enumerate(addrs, 1):
        if addr in cache:
            continue
        cache[addr] = kakao_geocode(addr)
        time.sleep(0.05)

        if i % 10 == 0 or i == total:
            print_progress(i, total)

    print()

    df.loc[mask, "addr_tmp"] = (
        df.loc[mask, key_cols]
        .astype(str)
        .agg(" ".join, axis=1)
        .map(normalize_addr)
    )

    df.loc[mask, ["좌표X", "좌표Y"]] = (
        df.loc[mask, "addr_tmp"]
        .map(cache)
        .apply(lambda x: pd.Series(x) if x else pd.Series([None, None]))
        .values
    )

    df.drop(columns="addr_tmp", inplace=True)

    return df




def fill_by_key_with_report(df, key_cols, cache, base_missing_cnt, step_name):
    start = time.perf_counter()

    mask = df["좌표X"].isna() | df["좌표Y"].isna()
    before_cnt = mask.sum()

    if before_cnt == 0:
        print(f"\n[{step_name}] 대상 없음 → 스킵")
        return df

    sub = df.loc[mask, key_cols].dropna().drop_duplicates()

    sub["addr"] = (
        sub[key_cols]
        .astype(str)
        .agg(" ".join, axis=1)
        .map(normalize_addr)
    )

    addrs = sub["addr"].dropna().unique()
    total_targets = len(addrs)

    if total_targets == 0:
        print(f"[{step_name}] 처리 대상 주소 없음")
        return df

    probe_n = min(100, total_targets)
    probe_start = time.perf_counter()

    for addr in addrs[:probe_n]:
        if addr not in cache:
            cache[addr] = kakao_geocode(addr)
            time.sleep(0.05)

    probe_elapsed = time.perf_counter() - probe_start
    est_total_time = probe_elapsed / probe_n * total_targets

    print(
        f"\n[{step_name}] 예상 소요 시간 : "
        f"{est_total_time/60:.1f} 분 "
        f"(총 {total_targets:,}건)"
    )

    api_calls = 0
    for addr in addrs:
        if addr in cache:
            continue

        cache[addr] = kakao_geocode(addr)
        api_calls += 1
        time.sleep(0.05)

        if api_calls % 10 == 0 or api_calls == total_targets:
            print_progress(api_calls, total_targets)

    print()

    df.loc[mask, "addr_tmp"] = (
        df.loc[mask, key_cols]
        .astype(str)
        .agg(" ".join, axis=1)
        .map(normalize_addr)
    )

    df.loc[mask, ["좌표X", "좌표Y"]] = (
        df.loc[mask, "addr_tmp"]
        .map(cache)
        .apply(lambda x: pd.Series(x) if x else pd.Series([None, None]))
        .values
    )

    df.drop(columns="addr_tmp", inplace=True)

    after_cnt = (df["좌표X"].isna() | df["좌표Y"].isna()).sum()
    solved = before_cnt - after_cnt

    elapsed = time.perf_counter() - start

    step_rate = solved / before_cnt * 100
    total_rate = (base_missing_cnt - after_cnt) / base_missing_cnt * 100

    print(f"API 호출 수      : {api_calls:,}")
    print(f"단계 해결 수     : {solved:,}")
    print(f"단계 해결률     : {step_rate:.2f}%")
    print(f"누적 해결률     : {total_rate:.2f}%")
    print(f"남은 미해결     : {after_cnt:,} ({100 - total_rate:.2f}%)")
    print(f"소요 시간       : {elapsed/60:.2f} 분")
    print()

    return df


def fill_road_with_buildingno(df, cache, base_missing_cnt):
    start = time.perf_counter()

    mask = (df["좌표X"].isna() | df["좌표Y"].isna())
    target = df.loc[mask, ["시군구", "도로명"]].dropna()

    target = target[
        target["도로명"].str.contains(r"\d", regex=True)
    ].drop_duplicates()

    addrs = (
        target[["시군구", "도로명"]]
        .astype(str)
        .agg(" ".join, axis=1)
        .map(normalize_addr)
        .unique()
    )

    total_targets = len(addrs)

    if total_targets == 0:
        print("\n[2단계: 도로명+건물번호] 대상 없음 → 스킵")
        return df

    print(
        f"\n[2단계: 도로명+건물번호] 예상 처리 건수 : {total_targets:,}"
    )

    api_calls = 0
    for addr in addrs:
        if addr not in cache:
            cache[addr] = kakao_geocode(addr)
            api_calls += 1
            time.sleep(0.05)

    df.loc[mask, "addr_tmp"] = (
        df.loc[mask, ["시군구", "도로명"]]
        .astype(str)
        .agg(" ".join, axis=1)
        .map(normalize_addr)
    )

    df.loc[mask, ["좌표X", "좌표Y"]] = (
        df.loc[mask, "addr_tmp"]
        .map(cache)
        .apply(lambda x: pd.Series(x) if x else pd.Series([None, None]))
        .values
    )

    df.drop(columns="addr_tmp", inplace=True)

    after_cnt = (df["좌표X"].isna() | df["좌표Y"].isna()).sum()
    solved = base_missing_cnt - after_cnt

    elapsed = time.perf_counter() - start

    print(f"API 호출 수      : {api_calls:,}")
    print(f"누적 해결 수     : {solved:,}")
    print(f"잔여 미해결     : {after_cnt:,}")
    print(f"소요 시간       : {elapsed/60:.2f} 분")
    print()

    return df


# def run_geocoding_pipeline(df, name="DATASET"):
#     print(f"\n===== {name} 시작 =====")

#     base_missing = (df["좌표X"].isna() | df["좌표Y"].isna()).sum()
#     print(f"초기 좌표 결측 : {base_missing:,}")

#     if base_missing == 0:
#         print("좌표 결측 없음 → 종료")
#         return df

#     cache = {}

#     df = fill_by_key_with_report(
#         df, ["시군구", "도로명"], cache, base_missing,
#         "1단계: 시군구+도로명"
#     )

#     df = fill_road_with_buildingno(
#         df, cache, base_missing
#     )

#     df = fill_by_key_with_report(
#         df, ["시군구", "번지"], cache, base_missing,
#         "3단계: 시군구+번지"
#     )

#     df = fill_by_key_with_report(
#         df, ["시군구", "본번", "부번"], cache, base_missing,
#         "4단계: 시군구+본번·부번"
#     )

#     print(f"\n===== {name} 종료 =====")
#     return df
def run_geocoding_pipeline(df):
    print("===== 시작 =====")

    base_missing = (df["좌표X"].isna() | df["좌표Y"].isna()).sum()
    unique_missing = (
        df.loc[df["좌표X"].isna() | df["좌표Y"].isna(), ["시군구", "도로명"]]
        .dropna()
        .drop_duplicates()
        .shape[0]
    )

    print(f"전체 좌표 결측 : {base_missing:,}")
    print(f"고유 주소 결측 : {unique_missing:,}")

    cache = {}

    # 1단계
    df = fill_by_key(
        df,
        ["시군구", "도로명"],
        cache,
        "1단계: 시군구+도로명"
    )

    # 2단계 (동 제거)
    df = fill_by_key(
        df,
        ["시군구", "도로명"],
        cache,
        "2단계: 시군구+도로명 (동 제거 재시도)",
        transform=lambda r: f"{remove_dong(r['시군구'])} {r['도로명']}"
    )

    # 3단계
    df = fill_by_key(
        df,
        ["시군구", "번지"],
        cache,
        "3단계: 시군구+번지"
    )

    # 4단계
    df = fill_by_key(
        df,
        ["시군구", "본번", "부번"],
        cache,
        "4단계: 시군구+본번·부번"
    )

    remaining = (df["좌표X"].isna() | df["좌표Y"].isna()).sum()
    print(f"\n잔여 좌표 결측 : {remaining:,}")
    print("===== 종료 =====")

    return df



In [11]:
def drop_unresolved_rows(df):
    print("\n===== 좌표 미해결 행 드랍 =====")

    before_rows = len(df)

    unresolved_mask = df["좌표X"].isna() | df["좌표Y"].isna()
    unresolved_cnt = unresolved_mask.sum()

    print(f"드랍 대상 좌표 결측 행 : {unresolved_cnt:,}")
    print(f"전체 대비 비율       : {unresolved_cnt / before_rows * 100:.3f}%")

    df = df.loc[~unresolved_mask].copy()

    after_rows = len(df)

    print(f"드랍 후 전체 행 수   : {after_rows:,}")
    print("===== 드랍 완료 =====\n")

    return df


In [ ]:

dt_coord = run_geocoding_pipeline(dt)
dt_coord_removed = drop_unresolved_rows(dt_coord)

In [12]:

dt_test_coord = run_geocoding_pipeline(dt_test)
print(dt_test_coord.shape)


===== 시작 =====
전체 좌표 결측 : 6,562
고유 주소 결측 : 2,054

[1단계: 시군구+도로명]
- 처리 대상 고유 주소 수 : 2,054
- 예상 소요 시간 : 4.9 분
|██████████████████████████████| 100.00%

[2단계: 시군구+도로명 (동 제거 재시도)]
- 처리 대상 고유 주소 수 : 13
- 예상 소요 시간 : 0.0 분


[3단계: 시군구+번지]
- 처리 대상 고유 주소 수 : 13
- 예상 소요 시간 : 0.0 분


[4단계: 시군구+본번·부번]
- 처리 대상 고유 주소 수 : 3
- 예상 소요 시간 : 0.0 분


잔여 좌표 결측 : 3
===== 종료 =====
(9272, 51)


In [ ]:
save_path = "../data/dt_coord.csv"
dt_coord_removed.to_csv(save_path, index=False)
print(f"저장 완료: {save_path}")

save_path = "../data/dt_test_coord.csv"
dt_test_coord.to_csv(save_path, index=False)
print(f"저장 완료: {save_path}")